In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt

This is my (JC) contribution to this project. I've implemented the code for splitting the dataset into seperate datasets that we can then use for evaulation, and hence scoring of the pre-trained models we're attempting to use

In [2]:
# Define a function to categorize questions
def naive_categorize_question(example):
    question = example['question'].lower()
    if 'date' in question:
        return 'date'
    if 'during' in question:
        return 'during'
    if 'how are' in question:
        return 'how are'
    if 'how big' in question or 'how large' in question:
        return 'how big/size'
    if 'how many' in question or 'how much' in question:
        return 'how m/m'
    if 'how old' in question:
        return 'how old'
    if 'what time' in question:
        return 'what time'
    if 'what' in question or 'which' in question:
        return 'what'
    if 'when' in question:
        return 'when'
    if 'where' in question:
        return 'where'
    if 'who' in question:
        return 'who'
    if 'whom' in question:
        return 'whom'
    if 'why' in question:
        return 'why'
    else:
        return 'undefined'

In [3]:
# Load the SQuAD dataset
dataset = load_dataset('squad', split='validation')

# Add a new field for question categories
dataset = dataset.map(lambda entry: {"category": naive_categorize_question(entry)})

# Split the dataset into categories
categories = dataset.unique('category')
categorized_datasets = {category: dataset.filter(lambda example: example['category'] == category) for category in categories}
print(categorized_datasets)

# Print out the size of each categorized dataset
for category, subset in categorized_datasets.items():
    print(f"Category '{category}': {len(subset)} questions")

# If you want to save these subsets, you can do so like this:
# for category, subset in categorized_datasets.items():
#     subset.to_csv(f'squad_{category}.csv')

{'what': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 6595
}), 'where': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 464
}), 'who': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 1151
}), 'undefined': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 421
}), 'how m/m': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 771
}), 'during': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 172
}), 'when': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 710
}), 'date': Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'category'],
    num_rows: 60
}), 'how old': Dataset({
    features: ['id', 'title', '

In [4]:
import transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

from transformers import pipeline

bbc = AutoModelForQuestionAnswering.from_pretrained('bert-base-cased')
bbc_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

: 

In [48]:
from transformers.pipelines.pt_utils import KeyPairDataset
from datasets import load_metric
import numpy as np

metric = load_metric("squad")
# Function to calculate metrics
def compute_metrics(eval_preds):
    
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Pretrained models to evaluate
model_names = [
    'distilbert-base-cased',
    'albert-base-v2',
    'bert-base-cased'
]

# Evaluate each model
for model_name in model_names:
    print(f"Evaluating {model_name}...")
    
    # Load the model and tokenizer
    # model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create a QA pipeline
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
    # for category in categorized_datasets.keys():
    for i in ['how big/size', 'what time']:
        print(f"Starting eval on category: {category}")
        dataset = categorized_datasets[category]
        # Making predictions and evaluating
        
        predictions = []
        for entry in dataset:
            result = qa_pipeline(entry["question"], entry["context"])
            pred = {'id': entry['id'], 'prediction_text': result['answer']}
            predictions.append(pred)

        ref_ds = dataset.select_columns(['id', 'answers'])
        references = []
        for entry in ref_ds:
            answer = {'id': entry['id'], 'answers': entry['answers']}
            references.append(answer)

        results = metric.compute(predictions=predictions, references=references)

        print(f"Model: {model_name} Category: {category}")
        print(f"Exact Match (EM): {results['exact_match']:.2f}")
        print(f"F1 Score: {results['f1']:.2f}\n")

# You would need to implement the compute_f1 function based on the SQuAD evaluation script or import it if available.
# Also note that making predictions on the full dataset would be slow and resource-intensive;
# you might want to batch the predictions or use a subset of the data.

Evaluating distilbert-base-cased...


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting eval on category: how are
Model: distilbert-base-cased Category: how are
Exact Match (EM): 0.00
F1 Score: 4.91

Starting eval on category: how are
Model: distilbert-base-cased Category: how are
Exact Match (EM): 0.00
F1 Score: 4.91

Evaluating albert-base-v2...


Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting eval on category: how are
Model: albert-base-v2 Category: how are
Exact Match (EM): 0.00
F1 Score: 4.62

Starting eval on category: how are
Model: albert-base-v2 Category: how are
Exact Match (EM): 0.00
F1 Score: 4.62

Evaluating bert-base-cased...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting eval on category: how are
Model: bert-base-cased Category: how are
Exact Match (EM): 0.00
F1 Score: 6.59

Starting eval on category: how are
Model: bert-base-cased Category: how are
Exact Match (EM): 0.00
F1 Score: 6.59



In [ ]:
question_answerer = pipeline("question-answering", model=bbc, tokenizer=tokenizer)

print(question_answerer(question="question", context="context"))

{'score': 0.2803937792778015, 'start': 0, 'end': 7, 'answer': 'context'}


In [46]:
metric = load_metric("squad")
print(metric)

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   